In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("test_data_explore").master("local[*]").getOrCreate()

# Read and Check Schema and Some records

In [19]:
df = spark.read.format('csv').option("header","true").load('/home/jovyan/dags/input/data/Waterbase_v2021_1_T_WISE6_DisaggregatedData.csv')

In [4]:
df.printSchema()

root
 |-- monitoringSiteIdentifier: string (nullable = true)
 |-- monitoringSiteIdentifierScheme: string (nullable = true)
 |-- parameterWaterBodyCategory: string (nullable = true)
 |-- observedPropertyDeterminandCode: string (nullable = true)
 |-- observedPropertyDeterminandLabel: string (nullable = true)
 |-- procedureAnalysedMatrix: string (nullable = true)
 |-- resultUom: string (nullable = true)
 |-- phenomenonTimeSamplingDate: string (nullable = true)
 |-- sampleIdentifier: string (nullable = true)
 |-- resultObservedValue: string (nullable = true)
 |-- resultQualityObservedValueBelowLOQ: string (nullable = true)
 |-- procedureLOQValue: string (nullable = true)
 |-- parameterSampleDepth: string (nullable = true)
 |-- parameterSedimentDepthSampled: string (nullable = true)
 |-- parameterSpecies: string (nullable = true)
 |-- resultMoisture: string (nullable = true)
 |-- resultFat: string (nullable = true)
 |-- resultExtractableLipid: string (nullable = true)
 |-- resultLipid: stri

In [21]:
df.

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
df.count()

60013752

In [9]:
df.groupBy("monitoringSiteIdentifier","monitoringSiteIdentifierScheme","parameterWaterBodyCategory").count().show()

+------------------------+------------------------------+--------------------------+-----+
|monitoringSiteIdentifier|monitoringSiteIdentifierScheme|parameterWaterBodyCategory|count|
+------------------------+------------------------------+--------------------------+-----+
|             DK18-310-08|          eionetMonitoringS...|                        GW|  719|
|             DK18-380-01|          eionetMonitoringS...|                        GW|  335|
|            DK238-971-01|          euMonitoringSiteCode|                        GW|  537|
|            DK34-1650-01|          euMonitoringSiteCode|                        GW| 1525|
|           DK206-1684-01|          euMonitoringSiteCode|                        GW|  470|
|            DK66-1523-11|          eionetMonitoringS...|                        GW|  224|
|            DK86-2079-03|          euMonitoringSiteCode|                        GW|  443|
|            DK66-1527-05|          eionetMonitoringS...|                        GW|  227|

In [10]:
df.groupBy("monitoringSiteIdentifier","monitoringSiteIdentifierScheme","parameterWaterBodyCategory").count().count()

57454

# Check which columns can be used for this project

In [4]:
s_df = spark.read.format('csv').option("header","true").load('/home/jovyan/dags/input/data/Waterbase_v2021_1_S_WISE6_SpatialObject_DerivedData.csv')

In [12]:
s_df.count()

70815

In [13]:
s_df.printSchema()

root
 |-- countryCode: string (nullable = true)
 |-- thematicIdIdentifier: string (nullable = true)
 |-- thematicIdIdentifierScheme: string (nullable = true)
 |-- monitoringSiteIdentifier: string (nullable = true)
 |-- monitoringSiteIdentifierScheme: string (nullable = true)
 |-- monitoringSiteName: string (nullable = true)
 |-- waterBodyIdentifier: string (nullable = true)
 |-- waterBodyIdentifierScheme: string (nullable = true)
 |-- waterBodyName: string (nullable = true)
 |-- specialisedZoneType: string (nullable = true)
 |-- naturalAWBHMWB: string (nullable = true)
 |-- reservoir: string (nullable = true)
 |-- surfaceWaterBodyTypeCode: string (nullable = true)
 |-- subUnitIdentifier: string (nullable = true)
 |-- subUnitIdentifierScheme: string (nullable = true)
 |-- subUnitName: string (nullable = true)
 |-- rbdIdentifier: string (nullable = true)
 |-- rbdIdentifierScheme: string (nullable = true)
 |-- rbdName: string (nullable = true)
 |-- confidentialityStatus: string (nullable 

In [5]:
lt_df = s_df.filter("countryCode=='LT'")

In [6]:
se_df = s_df.filter("countryCode=='SE'")

In [21]:
se_df.count()

760

In [8]:
from pyspark.sql.functions import col
lt_df.filter('monitoringSiteIdentifier is  null').show(3,False,True)

-RECORD 0---------------------------------------------------
 countryCode                    | LT                        
 thematicIdIdentifier           | LT001                     
 thematicIdIdentifierScheme     | eionetGroundWaterBodyCode 
 monitoringSiteIdentifier       | null                      
 monitoringSiteIdentifierScheme | null                      
 monitoringSiteName             | null                      
 waterBodyIdentifier            | LT001                     
 waterBodyIdentifierScheme      | eionetGroundWaterBodyCode 
 waterBodyName                  | UPPER - MIDDLE DEVONIAN   
 specialisedZoneType            | groundWaterBody           
 naturalAWBHMWB                 | null                      
 reservoir                      | null                      
 surfaceWaterBodyTypeCode       | null                      
 subUnitIdentifier              | null                      
 subUnitIdentifierScheme        | null                      
 subUnitName            

In [38]:
s_df = spark.read.format('csv').option("header","true").load('/home/jovyan/dags/input/data/Waterbase_v2021_1_S_WISE6_SpatialObject_DerivedData.csv')

In [35]:
from pyspark.sql.functions import to_json, struct

In [36]:
(s_df.select(to_json(struct('*')).alias('value'))
 .write
  .format("kafka")
  .option("kafka.bootstrap.servers", "broker:29092")
  .option("topic", "test_air")
  .save())

Py4JJavaError: An error occurred while calling o341.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 21.0 failed 1 times, most recent failure: Lost task 1.0 in stage 21.0 (TID 62) (857968d6741a executor driver): java.lang.NoSuchMethodError: 'scala.Enumeration$Value org.apache.spark.sql.catalyst.expressions.Cast$.apply$default$4()'
	at org.apache.spark.sql.kafka010.KafkaRowWriter.createProjection(KafkaWriteTask.scala:128)
	at org.apache.spark.sql.kafka010.KafkaRowWriter.<init>(KafkaWriteTask.scala:76)
	at org.apache.spark.sql.kafka010.KafkaWriteTask.<init>(KafkaWriteTask.scala:41)
	at org.apache.spark.sql.kafka010.KafkaWriter$.$anonfun$write$1(KafkaWriter.scala:71)
	at org.apache.spark.sql.kafka010.KafkaWriter$.$anonfun$write$1$adapted(KafkaWriter.scala:70)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2(RDD.scala:1020)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2$adapted(RDD.scala:1020)
	at org.apache.spark.SparkContext.$an

In [11]:
import os
os.listdir('..')

['data_explore',
 'kafka_manager.ipynb',
 'data_processor.ipynb',
 'reader.ipynb',
 'output',
 '.ipynb_checkpoints',
 'data_handler',
 'task']